In [3]:
import pandas as pd
import sqlite3

# Define a simple linear regression function
def LinearRegressionFit(X, y):
    n = len(X)
    x_mean = sum(X) / n
    y_mean = sum(y) / n

    numerator = sum((X[i] - x_mean) * (y[i] - y_mean) for i in range(n))
    denominator = sum((X[i] - x_mean) ** 2 for i in range(n))

    b1 = numerator / denominator
    b0 = y_mean - b1 * x_mean

    return b0, b1

# Load your historical data up to 2020
data = pd.read_csv('C:/Users/PREDATOR/web/renewable.csv')

# Handle missing values (you can use imputation or drop rows with missing values)
data = data.dropna()

# Define the independent variables (features) and target variable
feature_columns = ['Electricity from hydro (TWh)', 'Electricity from wind (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)', 'Total (TWh)']

# Create a new DataFrame for the years 2021 to 2050
years = list(range(2021, 2051))

# Initialize a list to store predictions for each entity and each column
all_predictions = []

# Group the data by the 'Country' column and rename it to 'Entity'
grouped_data = data.groupby('Entity')

# Iterate through each entity group
for entity, entity_data in grouped_data:
    # Initialize dictionaries to store predictions for the current entity for each column
    predictions = {'Entity': entity}
    
    # Iterate through each column and perform linear regression
    for column in feature_columns:
        X = entity_data['Year'].tolist()
        y = entity_data[column].tolist()
        
        # Check if data for the year 2020 exists
        if (entity_data['Year'] == 2020).any():
            estimated_value_last_year = entity_data[entity_data['Year'] == 2020][column].values[0]
        else:
            # Handle the case when 2020 data is missing
            estimated_value_last_year = 0  # You can set this to any suitable value
        
        # Calculate b0 and b1 for the current entity and column
        b0, b1 = LinearRegressionFit(X, y)
        
        # Make predictions for the current entity and column for the years 2021 to 2050
        column_predictions = []
        for year in years:
            prediction = b0 + b1 * year
            if prediction < 0:
                prediction = 0

            # Append the prediction to the list of predictions for the current column
            column_predictions.append(prediction)
            
            # Update the estimated value for the next year
            estimated_value_last_year = prediction
        
        # Store the predictions for the current column
        predictions[column] = column_predictions
    
    # Store the predictions for the current entity
    all_predictions.append(predictions)

# Create a DataFrame from predictions
entity_list = []
year_list = []
hydro_list = []
wind_list = []
solar_list = []
bioenergy_list = []
total_list = []

for prediction_data in all_predictions:
    entity = prediction_data['Entity']
    for year, hydro, wind, solar, bioenergy, total in zip(
        years,
        prediction_data['Electricity from hydro (TWh)'],
        prediction_data['Electricity from wind (TWh)'],
        prediction_data['Electricity from solar (TWh)'],
        prediction_data['Other renewables including bioenergy (TWh)'],
        prediction_data['Total (TWh)'],
    ):
        entity_list.append(entity)
        year_list.append(year)
        hydro_list.append(hydro)
        wind_list.append(wind)
        solar_list.append(solar)
        bioenergy_list.append(bioenergy)
        total_list.append(total)

output_df = pd.DataFrame({
    'Entity': entity_list,
    'Year': year_list,
    'Electricity from hydro (TWh)': hydro_list,
    'Electricity from wind (TWh)': wind_list,
    'Electricity from solar (TWh)': solar_list,
    'Other renewables including bioenergy (TWh)': bioenergy_list,
    'Total (TWh)': total_list,
})

# Connect to SQLite database and store the DataFrame
conn = sqlite3.connect('renewable_predictions_test.db')
output_df.to_sql('predictions', conn, if_exists='replace', index=False)
conn.commit()
conn.close()
